In [1]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# DenseNet121 model for image identification
image_model = DenseNet121(weights='imagenet', include_top=False)

#pooling layer
x = image_model.output
x = GlobalAveragePooling2D()(image_model.output)
# fully-connected layer
x = Dense(1024, activation='relu')(x)
predictions = Dense(1000, activation='softmax')(x)

# Model to be trained
dnModel = Model(inputs=image_model.input, outputs=predictions)

# Freezing the image model layers incase further layers are needed
for layer in image_model.layers:
    layer.trainable = False

# model compilation
dnModel.compile(optimizer='rmsprop', loss='categorical_crossentropy')


In [2]:
#poem generation model

import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import random
import sys

# Load the Gutenberg dataset
df = pd.read_parquet('gutenburg.parquet')

# Remove lines that contain non-ASCII chars
df = df[df['line'].map(lambda x: x.isascii())]

# Join all lines into a single string of text
text = df['line'].str.cat(sep=' ')

# sorted list of unique chars and dicts mapping chars to indices and vice versa
chars = sorted(list(set(text)))
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}
seqlen = 30  # sequence length, reduce if training time becomes an issue

# create training data from gutenburg dataset
def generator(sentence_list, next_word_list, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, seqlen, len(chars)), dtype=bool)
        y = np.zeros((batch_size, len(chars)), dtype=bool)
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index % len(sentence_list)]):
                x[i, t, char_indices[w]] = 1
            y[i, char_indices[next_word_list[index % len(sentence_list)]]] = 1
            index = index + 1
        yield x, y
        
# declaration & loop for next chars and sentences
sentences = []
next_chars = []
step = seqlen
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen])
    next_chars.append(text[i + seqlen])

In [18]:
# LSTM model for poem generation
model = Sequential()
model.add(LSTM(120, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(LSTM(120, return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

In [19]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 30, 120)           105600    
                                                                 
 lstm_5 (LSTM)               (None, 30, 120)           115680    
                                                                 
 dense_4 (Dense)             (None, 30, 99)            11979     
                                                                 
Total params: 233259 (911.17 KB)
Trainable params: 233259 (911.17 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
# helper method for index sampling
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas) 

In [6]:
#generate progress text after each epoch
def on_epoch_end(epoch, _):
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)

    for diversity in [0.2, 0.5, 1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + seqlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, seqlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

#set batch size
batch_size = 1028

#train model
model.fit(generator(sentences, next_chars, batch_size),
          steps_per_epoch=int(len(sentences)/batch_size),
          epochs=50,
          callbacks=[print_callback])


Epoch 1/50


KeyboardInterrupt: 

In [10]:
#testing
# print out the size of the chars set
print(len(chars))

99


In [21]:
model.load_weights('./2_by_120')

In [22]:
start_index = random.randint(0, len(text) - seqlen - 1)

for diversity in [.1,.4,.5,.6,1.0]:
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + seqlen]
    sentence = "                    brown bear"
    sentence = sentence[0:seqlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(200):
        x_pred = np.zeros((1, seqlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], diversity)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

----- diversity: 0.1
----- Generating with seed: "                    brown bear"
                    brown beard and the stars of the stars of the stars of the stars of the stars of the seas of the stars of the stars of the seas of the seas of the stars of the stars of the stars of the stars of the state, And 
----- diversity: 0.4
----- Generating with seed: "                    brown bear"
                    brown bears, and the state of the single spell. The heart was fair to be so free, The one ready heart of the stars of the souls of the parting of the stars of strength and the spear, She seizes to see, the suns
----- diversity: 0.5
----- Generating with seed: "                    brown bear"
                    brown bearing into a part when the tales sing: While the strife of the walls and the trembling seas of sun, the summer talks of shadow; there is sense The shadows of the cares of seas the way, Some prize and th
----- diversity: 0.6
----- Generating with seed: "           

In [23]:
#poem generation model
# Load the Gutenberg dataset
# Data is re-loaded here to accommodate a different preprocessing form used before settling on the one above
df = pd.read_parquet('gutenburg.parquet')

# Remove lines that contain non-ASCII chars
df = df[df['line'].map(lambda x: x.isascii())]

chars = sorted(list(set(df['line'].str.cat(sep=' '))))
print(chars)

['\x01', '\x07', '\t', '\x1a', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']


In [25]:
ids = set()
# print each unique gutenberg id
for id in df['gutenberg_id']:
    ids.add(id)
print(sorted(ids))
print(len(ids))

# append each line of the randomly selected 

train_ids = random.choices(list(ids), k= 50)
print(train_ids)

t = df.loc[df['gutenberg_id'].isin(train_ids)]
print(t)

[19, 20, 26, 58, 109, 136, 151, 163, 207, 213, 214, 228, 230, 232, 246, 257, 258, 259, 261, 262, 263, 264, 266, 301, 304, 309, 312, 313, 315, 317, 323, 328, 348, 353, 390, 391, 392, 397, 400, 409, 413, 424, 438, 441, 442, 454, 458, 487, 574, 579, 591, 592, 594, 595, 596, 602, 610, 615, 617, 618, 651, 658, 679, 680, 691, 692, 703, 715, 772, 785, 791, 795, 835, 841, 845, 937, 941, 962, 981, 982, 995, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1019, 1020, 1021, 1030, 1031, 1034, 1035, 1040, 1041, 1042, 1045, 1054, 1057, 1062, 1141, 1151, 1165, 1166, 1186, 1199, 1211, 1229, 1238, 1246, 1247, 1279, 1280, 1287, 1304, 1317, 1321, 1322, 1333, 1358, 1365, 1381, 1382, 1383, 1393, 1418, 1459, 1469, 1471, 1505, 1506, 1525, 1543, 1544, 1545, 1567, 1568, 1612, 1645, 1664, 1719, 1727, 1728, 1731, 1745, 1746, 1830, 1847, 1852, 1855, 1894, 1919, 1924, 1934, 1953, 1958, 1962, 1974, 1978, 1995, 1996, 1997, 2002, 2003, 2008, 2080, 2136, 2151, 2161, 2199, 2294, 2303, 2304, 2334, 2378, 2381, 2383, 2388

In [26]:
# Join all lines into a single string of text
text = t['line'].str.cat(sep=' ')
print(len(text))

5289334


In [27]:
# sorted list of unique chars and dicts mapping chars to indices and vice versa
chars = sorted(list(set(df['line'].str.cat(sep=' ')))) # should be changed to set(df['line']) in order to allow for evaluation
print(len(chars))
char_indices = {c: i for i, c in enumerate(chars)}
indices_char = {i: c for i, c in enumerate(chars)}

99


In [28]:
seqlen = 60
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

x = np.zeros((len(sentences), seqlen, len(chars)), dtype=bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1

In [30]:
# Comparing evaluation metrics across several different architectures
import matplotlib.pyplot as plt

# create models for cross-comparison
model1 = Sequential()
model1.add(LSTM(120, input_shape=(seqlen, len(chars)), return_sequences=True))
model1.add(Dense(len(chars), activation='softmax'))

# compile the models
model1.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy'])

# test performance across the 150 poem example dataset
# 10 epochs each model
h1 = model1.fit(x,y,
         batch_size= batch_size,
         epochs=10,
         verbose= 0)

KeyboardInterrupt: 

In [ ]:
model1 = Sequential()
model1.add(LSTM(120, input_shape=(seqlen, len(chars)), return_sequences=True))
model1.add(LSTM(120, return_sequences=True))
model1.add(Dense(len(chars), activation='softmax'))

model1.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy'])

h2 = model1.fit(generator(sentences, next_chars, batch_size),
        steps_per_epoch=int(len(sentences)/batch_size),
         batch_size= batch_size,
         epochs=10,
         verbose= 0)

In [ ]:
model1 = Sequential()
model1.add(LSTM(64, input_shape=(seqlen, len(chars)), return_sequences=True))
model1.add(LSTM(64, return_sequences=True))
model1.add(Dense(len(chars), activation='softmax'))

model1.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy'])

h3 = model1.fit(x,y,
         batch_size= batch_size,
         epochs=10,
         verbose= 0)

In [ ]:
model1 = Sequential()
model1.add(LSTM(256, input_shape=(seqlen, len(chars)), return_sequences=True))
model1.add(LSTM(256, return_sequences=True))
model1.add(Dense(len(chars), activation='softmax'))

model1.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy'])

h4 = model1.fit(x,y,
         batch_size= batch_size,
         epochs=10,
         verbose= 0)

In [ ]:
# list all data in history
print(h1.history.keys())

In [ ]:
plt.plot(h1.history['accuracy'])
plt.plot(h2.history['accuracy'])
plt.plot(h3.history['accuracy'])
plt.plot(h4.history['accuracy'])
plt.title('Accuracy Comparison of Different Models')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['1x120_layer', '2x120_layers', '2x64_layers', '2x256_layers'], loc='upper left')
plt.show()


plt.savefig("accuracy_comparison")

In [ ]:
plt.plot(h1.history['loss'])
plt.plot(h2.history['loss'])
plt.plot(h3.history['loss'])
plt.plot(h4.history['loss'])
plt.title('Loss Comparison of Different Models')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['1x120_layer', '2x120_layers', '2x64_layers', '2x256_layers'], loc='upper left')
plt.show()

plt.savefig("loss_comparison")

In [ ]:
plt.plot(h1.history['categorical_crossentropy'])
plt.plot(h2.history['categorical_crossentropy'])
plt.plot(h3.history['categorical_crossentropy'])
plt.plot(h4.history['categorical_crossentropy'])
plt.title('Categorical Crossentropy Comparison of Different Models')
plt.ylabel('Cross Entropy')
plt.xlabel('epoch')
plt.legend(['1x120_layer', '2x120_layers', '2x64_layers', '2x256_layers'], loc='upper left')
plt.show()

plt.savefig("entropy_comparison")